In [3]:
import os

# change this
vid_file = 'starwars.mp4'

#dont need to change these
repo_path = os.path.dirname(os.getcwd())
vid_path = '/vid'
testfile = repo_path + vid_path + '/' + vid_file

In [10]:
import cv2
import numpy as np

def extract_dominant(pic_in, clusters):
    
    # normalize array to 0-1 instead of 0-255
    pic_norm = np.array(pic_in).astype('float32')/255
    
    # convert to LAB
    pic_lab = cv2.cvtColor(pic_norm, cv2.COLOR_BGR2LAB)
    
    # reshape - make it long and 2D
    pic_reshape = np.float32(pic_lab.reshape(-1,3))
    
    # cluster
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5000, 5)
    flags = cv2.KMEANS_PP_CENTERS

    _, labels, palette = cv2.kmeans(pic_reshape, clusters, None, criteria, 30, flags)
    _, counts = np.unique(labels, return_counts=True)

    # most dominant cluster
    biggest_cluster = palette[np.argmax(counts)]
    
    # reshape to 3 channels
    biggest_cluster = np.float32(biggest_cluster.reshape(1,1,3))

    # convert back to BGR
    biggest_cluster = cv2.cvtColor(biggest_cluster, cv2.COLOR_LAB2BGR)
    
    # de-normalize back to bits
    biggest_cluster *= [255,255,255]
    
    return np.rint(biggest_cluster)

In [11]:
def parse_to_array(vid, n):
    cap = cv2.VideoCapture(vid)
    fps = cap.get(cv2.CAP_PROP_FPS)
    f = 0
    result = []
    times = []
    while(True):
        grabbed, frame = cap.read()
        if not grabbed:
            break
        f+=1
        sec = float(f)/fps
        if f % n == 0:
            result.append(extract_dominant(frame, 8)[:3])
            times.append(sec)
    cap.release()
    return result, times

In [ ]:
# take video file and take every nth frame, grabbing dominant color from each
training = parse_to_array(testfile, 23)

In [ ]:
training[0]